In [1]:
import io
import base64
import glob
import os
import pandas as pd

import gzip
import json
import numpy as np
import torch
from IPython.display import HTML
from skvideo.io import FFmpegReader, ffprobe, vwrite
from torch.autograd import Variable
from ptcap.trainers import DataParallelWrapper

/home/farzaneh/venv/ptcap/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)


In [2]:
def open_annotation(path):
    if path.endswith("gz"):
        with gzip.open(path, "rb") as f:
            json = pd.read_json(f.read().decode("utf-8"))
    else:
        json = pd.read_json(path)
    return json

In [3]:
def open_video(fname, framerate, size):
 
    metadata = ffprobe(fname)
    duration = float(metadata['video']['@duration'])
    # Compute corresponding nb of frames
    nframes = int(duration * framerate)
    oargs = {
        "-r": "%d" % framerate,
        "-vframes": "%d" % nframes,
        "-s": "%dx%d" % (size[0], size[1])
    }
    # Open file
    reader = FFmpegReader(fname, inputdict={}, outputdict=oargs)
    video = []
    # Get frames until there is no more
    for frame in reader.nextFrame():
        video.append(frame)
    # Return as a numpy array
    return np.array(video)

In [4]:
import torchvision.transforms as transforms

class ImagenetPreprocessor(object):
    def __init__(self, scale=1.):
        self.scale = scale

    def __call__(self, x):
        x_array = np.array(x, "float32") / self.scale

        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

        x_tensor = torch.FloatTensor (x)/self.scale
        num_frame, w,h, ch = x_tensor.size()
        normalized = []
        for f in range(num_frame):

            normalized.append(normalize(x_tensor[f,:,:,:].permute(2,0,1)))
        gg= np.array(torch.stack(normalized, dim=0),"float32")
        return gg

In [5]:
from rtorchn.data.fileio import MpegReader

reader = MpegReader(framerate=12, size=(224, 224), keep_aspect_ratio=False)

In [6]:
from torchvision.models import vgg16_bn
video_preprocess = ImagenetPreprocessor()
vgg_encoder = vgg16_bn()
vgg_encoder = vgg_encoder.cuda()

import torch.nn as nn

relu = nn.ReLU()

In [7]:
vgg_encoder.classifier[0](feature.view(-1, 512*7*7))

NameError: name 'feature' is not defined

In [8]:
512 * 7 * 7 / 4096

6.125

In [10]:
for filename in os.listdir("/data/20bn-something-something-v2"):
    if filename.endswith(".webm"):
        
        video = reader.open("/data/20bn-something-something-v2/"+filename)
        
        # Preprocess
        video = video_preprocess(video)
        v = Variable(torch.from_numpy(video), volatile=True).cuda()
        feature = vgg_encoder.features(v)
        feature = relu(vgg_encoder.classifier[0](feature.view(-1, 512*7*7)))
        f = feature.cpu().data.numpy()
        np.savez_compressed('/data/imagenet_features/'+filename, f)
        print(filename)

85859.webm
144941.webm
86563.webm
114396.webm
66060.webm
51984.webm
65972.webm
125811.webm
146109.webm
40274.webm
96090.webm
213741.webm
89568.webm
160759.webm
195808.webm
31937.webm
186953.webm
160448.webm
220157.webm
59509.webm
21312.webm
79103.webm
120751.webm
121550.webm
175259.webm
156675.webm
20100.webm
186033.webm
58388.webm
112787.webm
118689.webm
123847.webm
165080.webm
24727.webm
99266.webm
84995.webm
131000.webm
16442.webm
145774.webm
145565.webm
101476.webm
9779.webm
19315.webm
22334.webm
8969.webm
179331.webm
96084.webm
142075.webm
160812.webm
142689.webm
146130.webm
112283.webm
91919.webm
94978.webm
143757.webm
56149.webm
6290.webm
206131.webm
206228.webm
156110.webm
99521.webm
20925.webm
144516.webm
99224.webm
85064.webm
73032.webm
217579.webm
165757.webm
161010.webm
56023.webm
155621.webm
113415.webm
112969.webm
168779.webm
215542.webm
143812.webm
196865.webm
17894.webm
102948.webm
220248.webm
179849.webm
211192.webm
66001.webm
69863.webm
211701.webm
4264.webm
10835.web

8156.webm
69320.webm
70300.webm
207216.webm
43213.webm
169297.webm
159116.webm
145393.webm
62157.webm
119333.webm
23638.webm
164030.webm
67392.webm
164098.webm
4795.webm
171062.webm
218302.webm
80870.webm
118435.webm
208717.webm
99875.webm
191899.webm
164460.webm
114942.webm
86439.webm
13102.webm
4796.webm
190518.webm
96966.webm
209429.webm
85427.webm
99564.webm
149802.webm
54400.webm
47625.webm
200453.webm
68538.webm
190242.webm
193080.webm
195595.webm
173412.webm
199890.webm
216883.webm
37120.webm
59118.webm
62015.webm
16838.webm
54162.webm
141097.webm
187195.webm
138849.webm
502.webm
108919.webm
214464.webm
12594.webm
142324.webm
128129.webm
93612.webm
147374.webm
60452.webm
100392.webm
20673.webm
20569.webm
217946.webm
87396.webm
219201.webm
109285.webm
165162.webm
10571.webm
110218.webm
217643.webm
15221.webm
28429.webm
156355.webm
89173.webm
150660.webm
158162.webm
42929.webm
14769.webm
168683.webm
217484.webm
74916.webm
105371.webm
68353.webm
53116.webm
8951.webm
110542.webm
704

14458.webm
119242.webm
25176.webm
109165.webm
110366.webm
194420.webm
174505.webm
184527.webm
177249.webm
171074.webm
26043.webm
177534.webm
61095.webm
171114.webm
210062.webm
153201.webm
48893.webm
96551.webm
197614.webm
77512.webm
2732.webm
132964.webm
49515.webm
104684.webm
192179.webm
58761.webm
184433.webm
113211.webm
68147.webm
4344.webm
26645.webm
180743.webm
48964.webm
5391.webm
512.webm
189850.webm
176012.webm
84442.webm
61334.webm
213881.webm
95815.webm
217700.webm
91064.webm
198757.webm
64271.webm
145360.webm
151981.webm
9719.webm
18466.webm
155435.webm
42584.webm
191331.webm
172688.webm
169052.webm
190099.webm
47122.webm
195371.webm
18730.webm
91299.webm
32877.webm
56253.webm
99725.webm
44845.webm
6263.webm
139077.webm
200528.webm
166152.webm
216587.webm
216868.webm
176980.webm
152795.webm
186355.webm
119490.webm
50029.webm
16868.webm
59839.webm
92280.webm
27952.webm
66947.webm
197518.webm
163577.webm
48462.webm
116678.webm
136121.webm
45052.webm
161058.webm
107355.webm
107

69807.webm
14968.webm
23188.webm
53561.webm
109561.webm
71676.webm
220825.webm
217542.webm
132556.webm
9308.webm
195018.webm
47820.webm
42784.webm
174084.webm
111246.webm
87709.webm
136972.webm
97313.webm
164685.webm
131295.webm
139417.webm
6052.webm
134286.webm
201288.webm
96606.webm
54915.webm
181972.webm
82963.webm
119542.webm
66497.webm
118273.webm
93754.webm
19158.webm
62245.webm
98126.webm
43763.webm
172552.webm
139526.webm
138837.webm
99503.webm
175454.webm
109422.webm
40340.webm
69908.webm
132288.webm
38067.webm
31297.webm
76646.webm
84967.webm
88259.webm
198054.webm
50079.webm
212868.webm
182944.webm
219754.webm
77818.webm
103128.webm
50071.webm
195637.webm
129491.webm
51849.webm
36755.webm
159476.webm
3801.webm
82176.webm
112901.webm
139359.webm
51954.webm
204724.webm
118124.webm
9538.webm
174676.webm
86379.webm
95072.webm
87335.webm
78232.webm
165276.webm
205191.webm
20881.webm
58218.webm
45509.webm
99122.webm
138977.webm
163623.webm
54669.webm
192001.webm
114843.webm
182091

65845.webm
121099.webm
179280.webm
161582.webm
1633.webm
17181.webm
92438.webm
94804.webm
194645.webm
93917.webm
165501.webm
47303.webm
17513.webm
191735.webm
6107.webm
55628.webm
203442.webm
219532.webm
138584.webm
130380.webm
193574.webm
192828.webm
160362.webm
81235.webm
208590.webm
17407.webm
213852.webm
79869.webm
181370.webm
169623.webm
78786.webm
162602.webm
72290.webm
75105.webm
76353.webm
67308.webm
133839.webm
205508.webm
48220.webm
3313.webm
11445.webm
184331.webm
161750.webm
56352.webm
179036.webm
154120.webm
79053.webm
178595.webm
208485.webm
4091.webm
154867.webm
199822.webm
21008.webm
130875.webm
114117.webm
53447.webm
56493.webm
44360.webm
197114.webm
125943.webm
161770.webm
62936.webm
35656.webm
118779.webm
183935.webm
59623.webm
90239.webm
101340.webm
196010.webm
218451.webm
168763.webm
160832.webm
127481.webm
47823.webm
38204.webm
4505.webm
38498.webm
139292.webm
112031.webm
48056.webm
62344.webm
200305.webm
200229.webm
71476.webm
48794.webm
214596.webm
178942.webm
1

87744.webm
110278.webm
177682.webm
5307.webm
66018.webm
219913.webm
187656.webm
145786.webm
137731.webm
114457.webm
177697.webm
118888.webm
140175.webm
93394.webm
56439.webm
105891.webm
34646.webm
120822.webm
147773.webm
133153.webm
109213.webm
3519.webm
79901.webm
36703.webm
218345.webm
165011.webm
73966.webm
145528.webm
33337.webm
208321.webm
59454.webm
182217.webm
22791.webm
138284.webm
134108.webm
109962.webm
9603.webm
213684.webm
150275.webm
194135.webm
23192.webm
148777.webm
67770.webm
63556.webm
161077.webm
34733.webm
160951.webm
4524.webm
69190.webm
35433.webm
207947.webm
86529.webm
94282.webm
152871.webm
141566.webm
35240.webm
74989.webm
71423.webm
194669.webm
136907.webm
86733.webm
93572.webm
138526.webm
187510.webm
24190.webm
67058.webm
47438.webm
99980.webm
75371.webm
15865.webm
192692.webm
194401.webm
16287.webm
168274.webm
135902.webm
25492.webm
139543.webm
53329.webm
141922.webm
185632.webm
77861.webm
194690.webm
155483.webm
117019.webm
184489.webm
154483.webm
32047.webm

166895.webm
8302.webm
113837.webm
57895.webm
184821.webm
136181.webm
111697.webm
107464.webm
73641.webm
213111.webm
178512.webm
214505.webm
38378.webm
156722.webm
118693.webm
145550.webm
80780.webm
51302.webm
99961.webm
126480.webm
141428.webm
116609.webm
128509.webm
165424.webm
47164.webm
150137.webm
128631.webm
34444.webm
213612.webm
151247.webm
39611.webm
168884.webm
20678.webm
195558.webm
18331.webm
29930.webm
72474.webm
130523.webm
144662.webm
97258.webm
183525.webm
26411.webm
137022.webm
120945.webm
176559.webm
199727.webm
129417.webm
38518.webm
196947.webm
191381.webm
53113.webm
139418.webm
155141.webm
1123.webm
70129.webm
140580.webm
67117.webm
53446.webm
168105.webm
132784.webm
48701.webm
91561.webm
53726.webm
42049.webm
34865.webm
146601.webm
185443.webm
210969.webm
33893.webm
123028.webm
21956.webm
91925.webm
95998.webm
190183.webm
180853.webm
130011.webm
49428.webm
212102.webm
40304.webm
29497.webm
214644.webm
212082.webm
88948.webm
154804.webm
116971.webm
97747.webm
165517

172495.webm
154897.webm
107049.webm
201450.webm
28573.webm
114162.webm
135793.webm
166335.webm
47092.webm
134064.webm
173055.webm
30589.webm
35093.webm
92203.webm
123556.webm
146040.webm
39256.webm
215081.webm
5230.webm
173022.webm
124413.webm
33750.webm
71224.webm
137205.webm
177419.webm
72494.webm
35911.webm
92332.webm
65485.webm
173261.webm
163769.webm
93965.webm
91837.webm
89890.webm
176152.webm
210705.webm
77643.webm
213427.webm
42764.webm
60000.webm
3781.webm
128203.webm
38374.webm
161000.webm
88012.webm
216454.webm
93797.webm
219608.webm
201741.webm
18653.webm
53130.webm
161437.webm
125103.webm
123926.webm
218500.webm
146369.webm
864.webm
191495.webm
116160.webm
75887.webm
35669.webm
17423.webm
195436.webm
68212.webm
216276.webm
168449.webm
204353.webm
113108.webm
161497.webm
152920.webm
177399.webm
168657.webm
22265.webm
155013.webm
55256.webm
124994.webm
193031.webm
92334.webm
62888.webm
148535.webm
70782.webm
95130.webm
203422.webm
6654.webm
84923.webm
114955.webm
154126.webm

66188.webm
19501.webm
179119.webm
219365.webm
2989.webm
7468.webm
14002.webm
84489.webm
192251.webm
62969.webm
75775.webm
17166.webm
8623.webm
118374.webm
150661.webm
47727.webm
40667.webm
199872.webm
130552.webm
160299.webm
93595.webm
185224.webm
72070.webm
88218.webm
144780.webm
201225.webm
199502.webm
202994.webm
23646.webm
53248.webm
94761.webm
111355.webm
16861.webm
89123.webm
76699.webm
201906.webm
206696.webm
166452.webm
9876.webm
183647.webm
70789.webm
115948.webm
155551.webm
6496.webm
29626.webm
157254.webm
91708.webm
97289.webm
178780.webm
44455.webm
20094.webm
149829.webm
121479.webm
177463.webm
219897.webm
178809.webm
98939.webm
26938.webm
73901.webm
174179.webm
139700.webm
42446.webm
158795.webm
146408.webm
27008.webm
201396.webm
154236.webm
185212.webm
121049.webm
55447.webm
20572.webm
30188.webm
94546.webm
209092.webm
66566.webm
77709.webm
161719.webm
83702.webm
87250.webm
107779.webm
129734.webm
12698.webm
197106.webm
144315.webm
11545.webm
7661.webm
145743.webm
78162.w

ValueError: No way to determine width or height from video. Need `-s` in `inputdict`. Consult documentation on I/O.

In [26]:
!ls -la /data/imagenet_features/

total 252
drwxrwxr-x 2 farzaneh farzaneh  12288 Feb 16 16:42 .
drwxr-xr-x 7 farzaneh farzaneh   4096 Feb 14 18:36 ..
-rw-rw-r-- 1 farzaneh farzaneh 237849 Feb 16 16:42 85859.webm.npz


In [71]:
video = reader.open("/data/20bn-something-something-v2/"+filename)

In [72]:
video = video_preprocess(video)

In [73]:
v = Variable(torch.from_numpy(video)).cuda()

In [74]:
vgg_encoder = vgg_encoder.cuda()

In [77]:
feature = vgg_encoder.features(v)

In [96]:
feature.mean(dim=3).mean(dim=2).size()

torch.Size([27, 512])

In [81]:
f = feature.cpu().data.numpy()

In [85]:
np.savez_compressed('/home/farzaneh/abc.npy', f)

In [52]:
video.shape

(27, 224, 224, 3)